# 311 San Francisco Call Analysis

### Importation of Relevant Packages

In [1]:
from datetime import datetime
from sodapy import Socrata
import pandas as pd
import numpy as np
import requests

In [2]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.svm import SVC
from nltk import word_tokenize
from collections import defaultdict

In [11]:
!pip install lib

  Created wheel for lib: filename=lib-3.0.0-py3-none-any.whl size=3212 sha256=2be8b2d28fa0fe837d78c0bb3aef86e96c92356596d412049f453484e35def9c
  Stored in directory: /Users/jaklynsoler/Library/Caches/pip/wheels/a8/9f/c8/a1329ec3dd3d6214178425ade3a4a701b0ea521cb54e95388f
Successfully built lib


In [3]:
import nltk
from nltk.corpus import stopwords
import string
from nltk import word_tokenize, FreqDist
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.datasets import fetch_20newsgroups
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
import pandas as pd
import numpy as np
np.random.seed(0)
from nltk import word_tokenize
from collections import defaultdict
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.utils import to_categorical
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split

### Importing the Pickled Dataframe

In [4]:
df = pd.read_pickle("data/before_df.pkl")

### Observing the Dataframe

In [5]:
df.head()

,service_request_id,Date,agency_responsible,service_name,service_subtype,service_details,address,supervisor_district,neighborhoods_sffind_boundaries,police_district
0,13333616,2021-01-06,DPH - Other - G,General Request - DPH,request_for_service,dph_other - request_for_service,Intersection of TURK ST and JONES ST,6,Tenderloin,TENDERLOIN
1,13333606,2021-01-06,Mayors Office of Neighborhood Services - G,General Request - MONS,complaint,mons - complaint,Not associated with a specific address,NaN,NaN,NaN
2,13333604,2021-01-06,Parking Enforcement Dispatch Queue,Parking Enforcement,Parking_on_Sidewalk,Bronze - Lexus - N/a,"176 LEE AVE, SAN FRANCISCO, CA, 94112",11,Ingleside,TARAVAL
3,13333603,2021-01-06,Parking Enforcement Dispatch Queue,Parking Enforcement,Parking_on_Sidewalk,Parking Enforcement,"196 LEE AVE, SAN FRANCISCO, CA, 94112",11,Ingleside,TARAVAL
4,13333602,2021-01-06,Parking Enforcement Dispatch Queue,Parking Enforcement,Parking_on_Sidewalk,Parking Enforcement,"229 LEE AVE, SAN FRANCISCO, CA, 94112",7,Ingleside,TARAVAL


### Checking Dataframe Size

In [6]:
df.size

24193480

### Checking for NaN Values

In [7]:
df.isna().sum()

service_request_id                     0
Date                                   0
agency_responsible                     0
service_name                           0
service_subtype                        0
service_details                     1779
address                                1
supervisor_district                87577
neighborhoods_sffind_boundaries    88880
police_district                    89937
dtype: int64

### Dropping NaN Values - VARIABLE

In [8]:
df.dropna(inplace = True)

### NaN Values Removed

In [9]:
df.isna().sum()

service_request_id                 0
Date                               0
agency_responsible                 0
service_name                       0
service_subtype                    0
service_details                    0
address                            0
supervisor_district                0
neighborhoods_sffind_boundaries    0
police_district                    0
dtype: int64

### Observing Agencies

In [10]:
df.agency_responsible.value_counts()

DPW Ops Queue                                         865189
Recology_Abandoned                                    409184
Parking Enforcement Dispatch Queue                    154040
DPT Abandoned Vehicles Work Queue                      97109
HSOC Queue                                             74577
                                                       ...  
Mayors Office of Criminal Justice - G                      1
PUC - SCSR Queue                                           1
HSOC Review Queue                                          1
DPW - Bureau of Street and Sewer Repair - G - Hold         1
PUC - Electric/Power - G - Hold                            1
Name: agency_responsible, Length: 297, dtype: int64

### Observing Service Details

In [11]:
df.service_details.value_counts()

Other Loose Garbage                  298796
Encampment Cleanup                   213533
Boxed or Bagged Items                139306
Furniture                            129897
Human or Animal Waste                108789
                                      ...  
Blue - Toyota - 7SVJ604                   1
Black - Lexus Sedan - Unread              1
blue/silver - Honda - 5DLP972             1
White - Budget box truck - 3ak903         1
Black - Toyota Rav4 - 55529E2             1
Name: service_details, Length: 167705, dtype: int64

### Counting Agency Quantities

In [23]:
counter = Counter(df['agency_responsible'].tolist())

### Checking Agency Counts

In [24]:
counter

Counter({'DPH - Other - G': 524,
         'Parking Enforcement Dispatch Queue': 154040,
         'Recology_Abandoned': 409184,
         'DPT - Traffic Signal - G': 2270,
         'DPW Ops Queue': 865189,
         'DBI Housing Inspection Queue': 4033,
         'RPD NSA Queue': 7734,
         'Clear Channel - Transit Queue': 31545,
         'Recology_Overflowing': 57439,
         'DPH - Environmental Health - Noise Queue': 1146,
         'PUC Streetlights Queue': 11355,
         'Noise Report Queue': 10227,
         'DPT SignShop Repair Queue': 1611,
         'Muni Feedback Received Queue': 10586,
         'RPD Park Service Area 4 Queue': 2207,
         'PUC - Water - G': 11188,
         'Duplicate Case Hold Queue': 56865,
         'DPW BSM Queue': 53929,
         'AT and T - Graffiti Queue': 8475,
         'US Postal Service Maintenance Queue': 19479,
         'DPT SignShop Queue': 20294,
         'PUC Sewer Ops': 33476,
         'DPH - Environmental Health - G': 10624,
         'DBI Bu

### Finding Top 100 Agencies

In [25]:
top_agencies = {i[0]: idx for idx, i in enumerate(counter.most_common(100))}

### List of Top Agencies

The number is an index number, not the quantity that correlates to the requests.

In [26]:
top_agencies

{'DPW Ops Queue': 0,
 'Recology_Abandoned': 1,
 'Parking Enforcement Dispatch Queue': 2,
 'DPT Abandoned Vehicles Work Queue': 3,
 'HSOC Queue': 4,
 'Recology_Overflowing': 5,
 'Duplicate Case Hold Queue': 6,
 'DPW BSM Queue': 7,
 '311 Supervisor Queue': 8,
 'MUNI Work Queue': 9,
 'PUC Sewer Ops': 10,
 'Clear Channel - Transit Queue': 11,
 'DPT Meter_Bike Queue': 12,
 'Parking Enforcement Review Queue': 13,
 'Police - Homeless Concerns Queue': 14,
 'DPT SignShop Queue': 15,
 'US Postal Service Maintenance Queue': 16,
 'DPW BSSR Queue': 17,
 'DPW - Bureau of Street Environmental Services - G': 18,
 'Recology_Litter': 19,
 'PUC Streetlights Queue': 20,
 'PUC - Water - G': 21,
 'DPH - Environmental Health - G': 22,
 'Muni Feedback Received Queue': 23,
 'Noise Report Queue': 24,
 'SFMTA - Parking Enforcement - G': 25,
 'PG and E - Streetlights Queue': 26,
 'AT and T - Graffiti Queue': 27,
 'RPD NSA Queue': 28,
 'DPW - Bureau of Street Use and Mapping - G': 29,
 'Housing Authority SR Queue'

### i dont know what this part of the code is doing

In [27]:
df = df[df['agency_responsible'].map(lambda x: x in top_agencies)]

In [28]:
df.head()

,service_request_id,Date,agency_responsible,service_name,service_subtype,service_details,address,supervisor_district,neighborhoods_sffind_boundaries,police_district
0,13333616,2021-01-06,DPH - Other - G,General Request - DPH,request_for_service,dph_other - request_for_service,Intersection of TURK ST and JONES ST,6,Tenderloin,TENDERLOIN
2,13333604,2021-01-06,Parking Enforcement Dispatch Queue,Parking Enforcement,Parking_on_Sidewalk,Bronze - Lexus - N/a,"176 LEE AVE, SAN FRANCISCO, CA, 94112",11,Ingleside,TARAVAL
3,13333603,2021-01-06,Parking Enforcement Dispatch Queue,Parking Enforcement,Parking_on_Sidewalk,Parking Enforcement,"196 LEE AVE, SAN FRANCISCO, CA, 94112",11,Ingleside,TARAVAL
4,13333602,2021-01-06,Parking Enforcement Dispatch Queue,Parking Enforcement,Parking_on_Sidewalk,Parking Enforcement,"229 LEE AVE, SAN FRANCISCO, CA, 94112",7,Ingleside,TARAVAL
5,13333601,2021-01-06,Recology_Abandoned,Street and Sidewalk Cleaning,Bulky Items,Refrigerator,Intersection of 24TH ST and OSAGE ALY,9,Mission,MISSION


In [29]:
df.size

23191820

### List of Service Details

In [30]:
description_list = df['service_details'].tolist()

### Array of Agencies

In [31]:
agency_list = [top_agencies[i] for i in df['agency_responsible'].tolist()]
agency_list = np.array(agency_list)

In [32]:
agency_list

array([91,  2,  2, ...,  1,  0,  5])

In [33]:
count_vect = CountVectorizer()
x_train_counts = count_vect.fit_transform(description_list)

tfidf_transformer = TfidfTransformer()
x_train_tfidf = tfidf_transformer.fit_transform(x_train_counts)

train_x, test_x, train_y, test_y = train_test_split(x_train_tfidf, agency_list, test_size=0.3)


In [34]:
clf = MultinomialNB().fit(train_x, train_y)
y_score = clf.predict(test_x)

n_right = 0
for i in range(len(y_score)):
    if y_score[i] == test_y[i]:
        n_right += 1

print("Accuracy: %.2f%%" % ((n_right/float(len(test_y)) * 100)))

Accuracy: 77.45%


### Observing Accuracy Without Text Processing

The accuracy is lower than it could be with modeling. 

### Cleaning the Data

Creating functions to count the top words, replace them, and filter them. 

In [35]:
def count_top_x_words(corpus, top_x, skip_top_n):
    ''' docstring for counting words '''
    count = defaultdict(lambda: 0)
    for c in corpus:
        for w in word_tokenize(c):
            count[w] += 1
    count_tuples = sorted([(w, c) for w, c in count.items()], key=lambda x: x[1], reverse=True)
    return [i[0] for i in count_tuples[skip_top_n: skip_top_n + top_x]]


def replace_top_x_words_with_vectors(corpus, top_x):
    ''' replacing all the top words with vectors '''
    topx_dict = {top_x[i]: i for i in range(len(top_x))}

    return [
        [topx_dict[w] for w in word_tokenize(s) if w in topx_dict]
        for s in corpus
    ], topx_dict


def filter_to_top_x(corpus, n_top, skip_n_top=0):
    ''' filter words to top '''
    top_x = count_top_x_words(corpus, n_top, skip_n_top)
    return replace_top_x_words_with_vectors(corpus, top_x)

In [36]:
def count_top_x_words(corpus, top_x, skip_top_n):
    ''' '''
    count = defaultdict(lambda: 0)
    for c in corpus:
        for w in word_tokenize(c):
            count[w] += 1
    count_tuples = sorted([(w, c) for w, c in count.items()], key=lambda x: x[1], reverse=True)
    return [i[0] for i in count_tuples[skip_top_n: skip_top_n + top_x]]


def replace_top_x_words_with_vectors(corpus, top_x):
    ''' '''
    topx_dict = {top_x[i]: i for i in range(len(top_x))}

    return [
        [topx_dict[w] for w in word_tokenize(s) if w in topx_dict]
        for s in corpus
    ], topx_dict


def filter_to_top_x(corpus, n_top, skip_n_top=0):
    ''' '''
    top_x = count_top_x_words(corpus, n_top, skip_n_top)
    return replace_top_x_words_with_vectors(corpus, top_x)


### Modeling the Data

Importing the same pickled dataframe. 

In [55]:
df = pd.read_pickle("data/before_df.pkl")

In [56]:
df.head()

,service_request_id,Date,agency_responsible,service_name,service_subtype,service_details,address,supervisor_district,neighborhoods_sffind_boundaries,police_district
0,13333616,2021-01-06,DPH - Other - G,General Request - DPH,request_for_service,dph_other - request_for_service,Intersection of TURK ST and JONES ST,6,Tenderloin,TENDERLOIN
1,13333606,2021-01-06,Mayors Office of Neighborhood Services - G,General Request - MONS,complaint,mons - complaint,Not associated with a specific address,NaN,NaN,NaN
2,13333604,2021-01-06,Parking Enforcement Dispatch Queue,Parking Enforcement,Parking_on_Sidewalk,Bronze - Lexus - N/a,"176 LEE AVE, SAN FRANCISCO, CA, 94112",11,Ingleside,TARAVAL
3,13333603,2021-01-06,Parking Enforcement Dispatch Queue,Parking Enforcement,Parking_on_Sidewalk,Parking Enforcement,"196 LEE AVE, SAN FRANCISCO, CA, 94112",11,Ingleside,TARAVAL
4,13333602,2021-01-06,Parking Enforcement Dispatch Queue,Parking Enforcement,Parking_on_Sidewalk,Parking Enforcement,"229 LEE AVE, SAN FRANCISCO, CA, 94112",7,Ingleside,TARAVAL


In [57]:
counter = Counter(df['agency_responsible'].tolist())
top_agencies = {i[0]: idx for idx, i in enumerate(counter.most_common(100))}
df = df[df['agency_responsible'].map(lambda x: x in top_agencies)]


In [58]:
description_list = df['agency_responsible'].tolist()

### Creation of Array and Word List

In [62]:
mapped_list, word_list = filter_to_top_x(description_list, 2500, 0)


In [64]:
agency_list_o = [top_agencies[i] for i in df['agency_responsible'].tolist()]
agency_list = to_categorical(agency_list_o)


In [65]:
max_review_length = 150

mapped_list = sequence.pad_sequences(mapped_list, maxlen=max_review_length)
train_x, test_x, train_y, test_y = train_test_split(mapped_list, agency_list, test_size=0.3)

max_review_length = 150

embedding_vector_length = 64
model = Sequential()

model.add(Embedding(2500, embedding_vector_length, input_length=max_review_length))
model.add(Conv1D(50, 5))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(max(agency_list_o) + 1, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_x, train_y, epochs=3, batch_size=64)

y_score = model.predict(test_x)
y_score = [[1 if i == max(sc) else 0 for i in sc] for sc in y_score]
n_right = 0
for i in range(len(y_score)):
    if all(y_score[i][j] == test_y[i][j] for j in range(len(y_score[i]))):
        n_right += 1

print("Accuracy: %.2f%%" % ((n_right/float(len(test_y)) * 100)))

Epoch 1/3
26243/26243 [==============================] - 483s 18ms/step - loss: 0.0974 - accuracy: 0.9793
Epoch 2/3
26243/26243 [==============================] - 480s 18ms/step - loss: 1.6113e-04 - accuracy: 1.0000
Epoch 3/3
26243/26243 [==============================] - 483s 18ms/step - loss: 3.3870e-04 - accuracy: 1.0000
Accuracy: 100.00%


### Support Vector Machine Model

### Importing Pickled Dataframe

In [66]:
df = pd.read_pickle("data/before_df.pkl")

In [67]:
df.head()

,service_request_id,Date,agency_responsible,service_name,service_subtype,service_details,address,supervisor_district,neighborhoods_sffind_boundaries,police_district
0,13333616,2021-01-06,DPH - Other - G,General Request - DPH,request_for_service,dph_other - request_for_service,Intersection of TURK ST and JONES ST,6,Tenderloin,TENDERLOIN
1,13333606,2021-01-06,Mayors Office of Neighborhood Services - G,General Request - MONS,complaint,mons - complaint,Not associated with a specific address,NaN,NaN,NaN
2,13333604,2021-01-06,Parking Enforcement Dispatch Queue,Parking Enforcement,Parking_on_Sidewalk,Bronze - Lexus - N/a,"176 LEE AVE, SAN FRANCISCO, CA, 94112",11,Ingleside,TARAVAL
3,13333603,2021-01-06,Parking Enforcement Dispatch Queue,Parking Enforcement,Parking_on_Sidewalk,Parking Enforcement,"196 LEE AVE, SAN FRANCISCO, CA, 94112",11,Ingleside,TARAVAL
4,13333602,2021-01-06,Parking Enforcement Dispatch Queue,Parking Enforcement,Parking_on_Sidewalk,Parking Enforcement,"229 LEE AVE, SAN FRANCISCO, CA, 94112",7,Ingleside,TARAVAL


In [ ]:

counter = Counter(df['agency_responsible'].tolist())
top_agencies = {i[0]: idx for idx, i in enumerate(counter.most_common(10))}
df = df[df['agency_responsible'].map(lambda x: x in top_agencies)]

df.service_details = df.service_details.astype(str)

description_list = df['service_details'].tolist()
agency_list = [top_agencies[i] for i in df['agency_responsible'].tolist()]
agency_list = np.array(agency_list)

count_vect = CountVectorizer()
x_train_counts = count_vect.fit_transform(description_list)


tfidf_transformer = TfidfTransformer()
x_train_tfidf = tfidf_transformer.fit_transform(x_train_counts)

train_x, test_x, train_y, test_y = train_test_split(x_train_tfidf, agency_list, test_size=0.3)

clf = SVC(kernel='linear').fit(train_x, train_y)
y_score = clf.predict(test_x)

n_right = 0
for i in range(len(y_score)):
    if y_score[i] == test_y[i]:
        n_right += 1

print("Accuracy: %.2f%%" % ((n_right/float(len(test_y)) * 100)))